In [16]:
import numpy as np
import qiskit.quantum_info
import scipy
import qiskit
import c2qa
import sys
from c2qa.operators import CVOperators
sys.path.append('./src')

import importlib
import lchs_bos
import lchs
importlib.reload(lchs_bos)
importlib.reload(lchs)
from lchs_bos import *
from lchs import *
np.set_printoptions(suppress=True)

In [17]:
nq = 2 ### Problem Dimension
a,b = 0.1, 0 ## Define A
T = 0.1  ## Define the time interval [0,T]
beta = 0.9 # 0< beta < 1
epsilon = 0.001 #0.05  epsilon for LCHS parameters, epislon is the error tolerance
debug = False
trunc_multiplier = 3
## automatic computation
cv_ops = CVOperators()
dim = 2**nq
rng = numpy.random.Generator(numpy.random.PCG64(79489489))
## Random numpy
nhat = cv_ops.get_N(2**nq).todense()
# nhat = qiskit_normal_order_switch(nhat)
A = a*nhat + 1j*b*nhat
L,H = cart_decomp(A)
print(f"Norm of A: {numpy.linalg.norm(A, ord=2)}, Norm of L: {numpy.linalg.norm(L, ord=2)}, Norm of H: {numpy.linalg.norm(H, ord=2)}")
## Random initial state
# u0 = numpy.matrix( numpy.array([0]*dim,dtype=complex) ).reshape(-1,1)
# u0[2,0] = 1
u0 = numpy.matrix( rng.random((dim,1)) ,dtype=complex) ## dtype complex is necessary for scipy to compute complex integral
u0 = u0/numpy.linalg.norm(u0, ord=2)
print(u0)

Norm of A: 0.3, Norm of L: 0.3, Norm of H: 0.0
[[0.12222154+0.j]
 [0.5912622 +0.j]
 [0.22553724+0.j]
 [0.76459392+0.j]]


## Scipy Solution

In [18]:
def ode_func_ho(t,u):
    return numpy.array(-A.dot(u).reshape(-1))[0]
spi_sol_ho = scipy.integrate.solve_ivp(ode_func_ho, [0,T],numpy.array(u0.reshape(-1))[0], method='RK45')
if numpy.linalg.norm(spi_sol_ho.y[:,-1].imag,ord=2) < 1e-12:
    spi_uT_ho = spi_sol_ho.y[:,-1].real
else:
    spi_uT_ho = spi_sol_ho.y[:,-1]
spi_uT_ho_norm = spi_uT_ho/numpy.linalg.norm(spi_uT_ho,ord=2)
numpy.linalg.norm(spi_uT_ho,ord=2), spi_uT_ho, spi_uT_ho_norm

(0.9782834101777667,
 array([0.12222154, 0.58537904, 0.2210713 , 0.74199676]),
 array([0.1249347 , 0.59837368, 0.22597879, 0.7584681 ]))

## LCHS Classical

In [19]:
## Solve homogenous part
exp_op, uT = class_lchs_tihs(A, u0, T, beta, epsilon, trunc_multiplier=trunc_multiplier, trotterLH=False, verbose=1)
uT = numpy.array(uT).reshape(-1)
if numpy.linalg.norm(uT.imag,ord=2) < 1e-12:
    uT = uT.real
uT_err = numpy.linalg.norm(uT - spi_uT_ho,ord=2)
uTnormed_err = numpy.linalg.norm(uT/numpy.linalg.norm(uT,ord=2) - spi_uT_ho_norm,ord=2)
print("  Homogeneous u(T)=", uT, "  Norm=", numpy.linalg.norm(uT,ord=2))
print("  SciPy Sol   u(T)=", spi_uT_ho, "  Norm=", numpy.linalg.norm(spi_uT_ho,ord=2))
print("\n")
print("  Homogeneous solution error u(T)         :", uT_err, "   Relative error:", uT_err/numpy.linalg.norm(spi_uT_ho,ord=2))
print("  Homogeneous solution error u(T),normed  :", uTnormed_err, "   Relative error:", uTnormed_err/numpy.linalg.norm(spi_uT_ho,ord=2))


# print("\n\nTests with Classical Subroutine (Homogeneous, w/ trotter)")
## Solve homogenous part
exp_op2, uT2 = class_lchs_tihs(A, u0, T, beta, epsilon, trunc_multiplier=trunc_multiplier, trotterLH=True,verbose=0)
uT2 = numpy.array(uT2).reshape(-1)
if numpy.linalg.norm(uT2.imag,ord=2) < 1e-12:
    uT2 = uT2.real
uT_err2 = numpy.linalg.norm(uT2 - spi_uT_ho,ord=2)
uTnormed_err2 = numpy.linalg.norm(uT2/numpy.linalg.norm(uT2,ord=2) - spi_uT_ho_norm,ord=2)
print("  Homogeneous solution error u(T)(trotter):", uT_err2, "   Relative error:", uT_err2/numpy.linalg.norm(spi_uT_ho,ord=2))
print("  Homogeneous solution error u(T),normed  :", uTnormed_err2, "   Relative error:", uTnormed_err2/numpy.linalg.norm(spi_uT_ho,ord=2))

  Preset parameters T =  0.1 beta =  0.9 epsilon =  0.001
  Truncation range [-K,K] K = 36.78794411714423
  Step size h1 = 12.262648039048075
  Number of nodes in [mh_1, (m+1)h_1] Q = 9
  Total number of nodes M = 54
  Truncation error bound = 2.458009236028988
  Quadrature error bound = 0.0003849254439855928
  Total error bound = 2.4583941614729734
  ||c||_1 = 2.00707157560424
  Homogeneous u(T)= [0.12202213 0.58445011 0.22075455 0.74111291]   Norm= 0.9769607208712742
  SciPy Sol   u(T)= [0.12222154 0.58537904 0.2210713  0.74199676]   Norm= 0.9782834101777667


  Homogeneous solution error u(T)         : 0.0013357349305655227    Relative error: 0.001365386468449672
  Homogeneous solution error u(T),normed  : 0.00019049065615785301    Relative error: 0.00019471929522267827
  Homogeneous solution error u(T)(trotter): 0.0013357349305655227    Relative error: 0.001365386468449672
  Homogeneous solution error u(T),normed  : 0.00019049065615785301    Relative error: 0.00019471929522267827


##  Qubit

In [20]:
print("\n\nTests with Quantum Subroutine (Qiskit API)")
quant_uT, lchs_circ_ho, circ_op_ho, quant_coeffs, quant_unitaries, quant_coeffs_unrot, quant_unitaries_unrot = quant_lchs_tihs(A, u0, T, beta, epsilon, 
                                            trunc_multiplier=trunc_multiplier, trotterLH=True,
                                            qiskit_api=True, verbose=1, debug=False, rich_return=True)
quant_uT = numpy.array(quant_uT).reshape(-1)
if numpy.linalg.norm(quant_uT.imag,ord=2) < 1e-12:
    quant_uT = quant_uT.real

quant_uT_err = numpy.linalg.norm(quant_uT - spi_uT_ho,ord=2)
quant_uTnormed_err = numpy.linalg.norm(quant_uT/numpy.linalg.norm(quant_uT,ord=2) - spi_uT_ho_norm,ord=2)
quantclass_uT_err = numpy.linalg.norm(quant_uT - uT,ord=2)  
quantclass_uTnormed_err = numpy.linalg.norm(quant_uT/numpy.linalg.norm(quant_uT,ord=2) - uT/numpy.linalg.norm(uT,ord=2),ord=2)
print("  - Homogeneous   u(T)=", quant_uT, "  Norm=", numpy.linalg.norm(quant_uT,ord=2))
print("  - SciPy Sol     u(T)=", spi_uT_ho, "  Norm=", numpy.linalg.norm(spi_uT_ho,ord=2))
print("  - Classical Sol u(T)=", uT, "  Norm=", numpy.linalg.norm(uT,ord=2))
print("  Compare to SciPy solution")
print("  - Homogeneous solution error u(T)         :", quant_uT_err, "   Relative error:", quant_uT_err/numpy.linalg.norm(spi_uT_ho,ord=2))
print("  - Homogeneous solution error u(T),normed  :", quant_uTnormed_err, "   Relative error:", quant_uTnormed_err/numpy.linalg.norm(spi_uT_ho,ord=2))
print("  Compare to LCHS Classical solution")
print("  - Homogeneous solution error u(T)         :", quantclass_uT_err, "   Relative error:", quantclass_uT_err/numpy.linalg.norm(uT,ord=2))
print("  - Homogeneous solution error u(T),normed  :", quantclass_uTnormed_err, "   Relative error:", quantclass_uTnormed_err/numpy.linalg.norm(uT,ord=2))




Tests with Quantum Subroutine (Qiskit API)
  Preset parameters T =  0.1 beta =  0.9 epsilon =  0.001
  Truncation range [-K,K] K = 36.78794411714423
  Step size h1 = 12.262648039048075
  Number of nodes in [mh_1, (m+1)h_1] Q = 9
  Total number of nodes M = 54
  Truncation error bound = 2.458009236028988
  Quadrature error bound = 0.0003849254439855928
  Total error bound = 2.4583941614729734
  ||c||_1 = 2.00707157560424
  LCU-Oracle: num_qubits_control= 6 num_qubits_op= 2


/Users/zhen002/miniconda3/envs/qbos/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/zhen002/miniconda3/envs/qbos/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/zhen002/VirtualMachine/Ubuntu 24.04.1 LTS/qla_lib/src/oracle_synth.py:38: RuntimeWarning: invalid value encountered in scalar divide
  tmp2 = matrix_su[1,0]/numpy.sin(theta/2)


  Number of Qubits: 8
  Transpiled LCU Circ Stats (Opt 0): OrderedDict({'cx': 13830, 'u': 10922})
    Circuit Depth (Opt 0): 22612
  Transpiled LCU Circ Stats (Opt 2): OrderedDict({'cx': 11022, 'u': 10623})
    Circuit Depth (Opt 2): 19846
  - Homogeneous   u(T)= [0.12202213 0.58445011 0.22075455 0.74111291]   Norm= 0.9769607208713565
  - SciPy Sol     u(T)= [0.12222154 0.58537904 0.2210713  0.74199676]   Norm= 0.9782834101777667
  - Classical Sol u(T)= [0.12202213 0.58445011 0.22075455 0.74111291]   Norm= 0.9769607208712742
  Compare to SciPy solution
  - Homogeneous solution error u(T)         : 0.0013357349304845936    Relative error: 0.0013653864683669463
  - Homogeneous solution error u(T),normed  : 0.0001904906561629568    Relative error: 0.00019471929522789534
  Compare to LCHS Classical solution
  - Homogeneous solution error u(T)         : 8.258888281857546e-14    Relative error: 8.453654384888775e-14
  - Homogeneous solution error u(T),normed  : 5.336726713782377e-15    Relat

## Qubit control Qumodes

In [21]:
print("\n\nTests with Qumode Subroutine (Bosonic Qiskit)")
bos_uT, bos_uT_array, lchs_circ_bosho_array, lchs_op_bosho, bos_coeffs, bos_thetas = bos_pro_long_lchs_tihs([a,b], nq, u0, T, beta, epsilon, 
                                            trunc_multiplier=trunc_multiplier, trotterLH=True,
                                            qiskit_api=True, verbose=1, debug=True)
bos_uT = bos_uT.reshape(-1)
if numpy.linalg.norm(bos_uT.imag,ord=2) < 1e-12:
    bos_uT = bos_uT.real
bos_uT_err = numpy.linalg.norm(bos_uT - spi_uT_ho,ord=2)
bos_uTnormed_err = numpy.linalg.norm(bos_uT/numpy.linalg.norm(bos_uT,ord=2) - spi_uT_ho_norm,ord=2)
print("  Homogeneous u(T)=", bos_uT, "  Norm=", numpy.linalg.norm(bos_uT,ord=2))
print("  SciPy Sol   u(T)=", spi_uT_ho, "  Norm=", numpy.linalg.norm(spi_uT_ho,ord=2))
print("  Homogeneous solution error u(T)        :", bos_uT_err, "   Relative error:", bos_uT_err/numpy.linalg.norm(spi_uT_ho,ord=2))
print("  Homogeneous solution error u(T),normed  :", bos_uTnormed_err, "   Relative error:", bos_uTnormed_err/numpy.linalg.norm(spi_uT_ho,ord=2))



Tests with Qumode Subroutine (Bosonic Qiskit)
  Preset parameters T =  0.1 beta =  0.9 epsilon =  0.001
  Truncation range [-K,K] K = 36.78794411714423
  Step size h1 = 12.262648039048075
  Number of nodes in [mh_1, (m+1)h_1] Q = 9
  Total number of nodes M = 54
  Truncation error bound = 2.458009236028988
  Quadrature error bound = 0.0003849254439855928
  Total error bound = 2.4583941614729734
  ||c||_1 = 2.00707157560424
LCU bos debug
  Coeff Reconstruction error = 0.0


AttributeError: 'CVCircuit' object has no attribute 'Reset'

In [ ]:
temp_uT = (bos_uT_array[0] - bos_uT_array[1]) + 1j*(bos_uT_array[2] - bos_uT_array[3])

print(numpy.linalg.norm(temp_uT - spi_uT_ho,ord=2))

In [ ]:
bos_coeffs

In [ ]:
bos_thetas

In [ ]:
lchs_circ_bosho_array[0].decompose().draw()